### 多层感知机

In [1]:
import torch
import numpy
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))  # 定义了一个特殊的 Module（块）
# 这是一个简单的单层神经网络，
# 一个线性层（20是输入维度，256是隐藏层的神经元个数）
# + 一个ReLU（激活函数）
# + 一个线性层（256是隐藏层的神经元个数，10s是输出维度）
# nn.Linear()会自动初始化权重

X = torch.rand(2,20) # 生成一个随机的input  
# 2是pn大小（批量大小），20是维度，也可以看作是2*20的矩阵

X2 = net(X)  # 将input放入模型中得到output

print(345)
print(X)
print(X2)  # 输出经过模型处理后的output


345
tensor([[0.0425, 0.3996, 0.0564, 0.7655, 0.1120, 0.1696, 0.5256, 0.5552, 0.5087,
         0.8599, 0.6043, 0.1248, 0.8323, 0.0841, 0.4176, 0.6518, 0.7897, 0.5848,
         0.1280, 0.0270],
        [0.4337, 0.6388, 0.4869, 0.7589, 0.4771, 0.0353, 0.7225, 0.6173, 0.4926,
         0.6505, 0.9257, 0.8357, 0.6483, 0.4285, 0.7880, 0.2571, 0.4744, 0.7820,
         0.7323, 0.2980]])
tensor([[ 0.2059,  0.0981, -0.2152,  0.2446, -0.0161, -0.0482, -0.1010, -0.2831,
          0.0262, -0.0369],
        [ 0.2450,  0.1175, -0.1669,  0.1156, -0.0504,  0.1044,  0.0087, -0.2527,
          0.2466, -0.0079]], grad_fn=<AddmmBackward0>)


### 自定义块

任何一个 层 或者 神经网络 都是 Module 的一个子类

In [2]:
class MLP(nn.Module):   # MLP继承自 Module
    # 一共有两个重要的函数
    # 一个是__init__函数，在里面定义我们需要用到的属性
    def __init__(self):    
        super().__init__()  # 继承父类的__init__，它会帮我们把一些参数都初始化好
        # 两个全连接层
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)    # 输出层
        
    # 前向传播函数
    def forward(self, X):   # X 输入的数据
        return self.out(F.relu(self.hidden(X)))
        # 先把input放到隐藏层里，这里的 hidden() 是继承自父类的一个动态方法，而不是我们定义的 hidden 属性
        # 然后调用 functional 的 relu函数（即激活函数）来激活
        # 然后放到输出函数里放回输出
        # 这个 out() 是继承自父类的一个动态方法
        
    # 上一个cell的nn.ReLU()是创建一个ReLU类型的对象并作为参数传入到nn.Sequential中进行使用，
    # 而这里的F.relu()是直接调用激活函数
    


### 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net2 = MLP()  
X2 = net2(X)
# 这里实例化MLP类后，net2()会自动调用继承自Module类的魔法方法__call__()，进而调用forward()方法
print(X)
print(X2)

tensor([[0.0425, 0.3996, 0.0564, 0.7655, 0.1120, 0.1696, 0.5256, 0.5552, 0.5087,
         0.8599, 0.6043, 0.1248, 0.8323, 0.0841, 0.4176, 0.6518, 0.7897, 0.5848,
         0.1280, 0.0270],
        [0.4337, 0.6388, 0.4869, 0.7589, 0.4771, 0.0353, 0.7225, 0.6173, 0.4926,
         0.6505, 0.9257, 0.8357, 0.6483, 0.4285, 0.7880, 0.2571, 0.4744, 0.7820,
         0.7323, 0.2980]])
tensor([[ 0.0847, -0.0005, -0.0721, -0.0458,  0.0151,  0.0037, -0.0317,  0.0987,
          0.0328,  0.0817],
        [ 0.1048,  0.1360, -0.1037, -0.0899,  0.0462,  0.1015, -0.1689,  0.1203,
          0.1553,  0.1069]], grad_fn=<AddmmBackward0>)


### 顺序快
模拟实现nn.Sequential()这个类

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):  
        # '*args'：list of inpt arguments，即一个收集参数的迭代器，
        # 相当于把若干个参数打包成一个来传入，然后依次被调用，是以字典的形式打包的
        super().__init__()
        for block in args:  
            # 依次调用传入的一个个参数，通过第一个cell里的代码可以知道，也即一个个类，也即一个个层
            self._modules[block] = block  
            # _modules是我们定义的一个特殊的成员变量，前面的下划线是为了命名冲突
            # 从这个写法可以看出，它其实是一个有序字典，我们把传进来每个层既作为key也作为value存进该字典中
            # 不过需要注意的是，value存的是它原来的类型，而key存的是它的字符串形式
        # 其实除了用dict来存，也可以直接用list
        # 另外，python3.6之后的dict默认是有序的
            
    def forward(self, X):
        for block in self._modules.values():  # 这里是要对values进行遍历，而不是keys
            # 每个block都是一个层，也即一个类
            X = block(X)  # 将传入的input（即X）依次放入层中进行处理，并替换原来的值
        return X    # 返回经过这些层处理过后的X
    # 注意，X是经过拷贝的，这里传进来的X是拷贝后的，类外的X仍然不变
    
net3 = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X2 = net2(X)
print(X)
print(X2)

tensor([[0.0425, 0.3996, 0.0564, 0.7655, 0.1120, 0.1696, 0.5256, 0.5552, 0.5087,
         0.8599, 0.6043, 0.1248, 0.8323, 0.0841, 0.4176, 0.6518, 0.7897, 0.5848,
         0.1280, 0.0270],
        [0.4337, 0.6388, 0.4869, 0.7589, 0.4771, 0.0353, 0.7225, 0.6173, 0.4926,
         0.6505, 0.9257, 0.8357, 0.6483, 0.4285, 0.7880, 0.2571, 0.4744, 0.7820,
         0.7323, 0.2980]])
tensor([[ 0.0847, -0.0005, -0.0721, -0.0458,  0.0151,  0.0037, -0.0317,  0.0987,
          0.0328,  0.0817],
        [ 0.1048,  0.1360, -0.1037, -0.0899,  0.0462,  0.1015, -0.1689,  0.1203,
          0.1553,  0.1069]], grad_fn=<AddmmBackward0>)


### 在正向传播函数中执行代码
以下只是一个例子，告诉我们可以自由地定义forwad()，而这个例子本身没有什么实际作用

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        # 这行代码创建了一个形状为 (20, 20) 的随机权重矩阵，
        # 并将其 requires_grad 属性设置为 False，意味着在反向传播时不会计算这个矩阵的梯度。
        # 这个矩阵在网络的前向传播中被用作一个固定的（不学习的）权重。
        self.linear = nn.Linear(20,20)
        
    # 对前向传播方法的自定义设计
    def forward(self, X):
        X = self.linear(X)  # X 先通过一个线性层
        X = F.relu(torch.mm(X, self.rand_weight) + 1)  # 再通过一个激活函数
        # X 与前面定义的固定权重矩阵 self.rand_weight 进行矩阵乘法，结果加上1，最后通过ReLU激活函数。
        # +1 相当于是做一下偏移吧
        # 这里使用 torch.mm 进行矩阵乘法，但更简洁的方式是直接使用 X @ self.rand_weight（若PyTorch版本支持）
        X = self.linear(X)  # 再通过一个线性层
        while X.abs().sum() > 1:  
            X /= 2
        # 不断将 X 除以2，直到 X 的绝对值之和不大于1。
        # 这是一个归一化步骤，但它与常见的归一化方法（如批归一化、层归一化等）不同，
        # 因为它是在网络内部动态调整的，并且是基于整个 X 的绝对值之和。    
        # 其实这个例子就是随便定义随便玩玩，没啥实际意义
        
        return X.sum()
    
    # 还有一个是反向传播，反向传播其实就是求导
    
net3 = FixedHiddenMLP()
X2 = net3(X)
print(X)
print(X2)

tensor([[0.0425, 0.3996, 0.0564, 0.7655, 0.1120, 0.1696, 0.5256, 0.5552, 0.5087,
         0.8599, 0.6043, 0.1248, 0.8323, 0.0841, 0.4176, 0.6518, 0.7897, 0.5848,
         0.1280, 0.0270],
        [0.4337, 0.6388, 0.4869, 0.7589, 0.4771, 0.0353, 0.7225, 0.6173, 0.4926,
         0.6505, 0.9257, 0.8357, 0.6483, 0.4285, 0.7880, 0.2571, 0.4744, 0.7820,
         0.7323, 0.2980]])
tensor(-0.2151, grad_fn=<SumBackward0>)


In [ ]:
### 混合搭配各种组合块的方法
以下就是一个随意嵌套的例子，也没有什么实际作用。只是告诉我们有多样的嵌套的可能。
不过要注意的是，这一层的输入维度与输出之间、层与层之间的维度应该相等

In [6]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(), nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP())
X2 = chimera(X)
print(X2)

tensor(-0.0065, grad_fn=<SumBackward0>)
